In [111]:
"""Convert a csv file to a table in a Microsoft Word document."""
import os
import numpy as np
import pandas as pd
import docx
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.enum.section import WD_ORIENT
from docx.shared import Inches

In [170]:
def format_number(x):
    """Check type of `x` and format it appropriately."""
    if isinstance(x, float):
        return f"{x:.3f}"
    elif isinstance(x, (int, np.integer)):
        return f"{x:,}"
    else:
        return str(x)

def add_asterisks_string(s, n):
    return s + '*' * n

def add_asterisks_column(df, col_number, n):
    f = lambda s: add_asterisks_string(s, n)
    df.iloc[:, col_number] = df.iloc[:, col_number].apply(f)

In [180]:
filename = "excel2.xlsx" # Change this. Should be the name of the file without file extension
directory = "private"
df = pd.read_excel(os.path.join(directory, filename), header=[0,1])
df = df.applymap(format_number) # Format the numbers
add_asterisks_column(df, 1, 3)
add_asterisks_column(df, 6, 3)
df

Unnamed: 0_level_0    HS GPA                                       \
   Unnamed: 0_level_1     Coeff Odds Ratio Eff Size   S.E. Pseudo R2   
0               Total  0.182***      1.199    0.071  0.032     0.228   
1                   A  0.172***      1.187    0.097  0.080     0.137   
2                   B  0.202***      1.224    0.080  0.133     0.198   
3                   C  0.168***      1.183    0.082  0.119     0.203   
4                   D  0.179***      1.196    0.067  0.093     0.117   
5                   E  0.188***      1.207    0.101  0.254     0.180   
6                   F  0.187***      1.206    0.047  0.093     0.162   
7                   G  0.224***      1.251    0.080  0.227     0.128   
8                   H  0.209***      1.232    0.126  0.147     0.231   
9                   I  0.198***      1.219    0.113  0.111     0.173   
10                  J  0.240***      1.272    0.105  0.181     0.253   
11                  K  0.291***      1.338    0.030  0.288     0.088   
12                  L  0.252***      1.287    0.080  0.269     0.157   
13                  M  0.150***      1.162    0.032  0.270     0.268   
14                  N  0.204***      1.226    0.099  0.121     0.222   
15                  O  0.130***      1.139    0.046  0.100     0.161   

     CTX GPA                                               
       Coeff Odds Ratio Eff Size   S.E. Pseudo R2       N  
0   1.175***      3.238    0.060  0.023     0.219  49,437  
1   1.202***      3.327    0.086  0.062     0.128   5,413  
2   1.269***      3.557    0.067  0.091     0.183   3,108  
3   1.190***      3.287    0.069  0.093     0.193   2,655  
4   1.222***      3.394    0.063  0.066     0.113   5,718  
5   1.463***      4.319    0.105  0.198     0.182     615  
6   1.028***      2.795    0.037  0.060     0.154   8,662  
7   1.604***      4.973    0.083  0.151     0.130   1,471  
8   1.501***      4.486    0.118  0.109     0.219   1,751  
9   1.364***      3.912    0.099  0.082     0.162   3,337  
10  1.621***      5.058    0.087  0.132     0.231   2,035  
11  0.856***      2.354    0.012  0.122     0.074   4,944  
12  1.435***      4.200    0.059  0.170     0.136   1,315  
13  0.948***      2.581    0.025  0.195     0.257   1,215  
14  1.128***      3.089    0.065  0.082     0.189   3,299  
15  1.013***      2.754    0.049  0.075     0.164   3,714

In [181]:
df.shape
df.iloc[15, 1]

'0.130***'

In [182]:
# Create an empty document
doc = docx.Document()

# Adjust margins
sections = doc.sections
for section in sections:
    section.left_margin = Inches(1)
    section.right_margin = Inches(1)
    section.orientation = WD_ORIENT.LANDSCAPE
    section.page_width = Inches(11)
    section.page_height = Inches(8.5)

ncolumns = df.shape[-1] # Number of columns
nlevels = df.columns.nlevels # Number of header rows
nrows = df.shape[0] # Number of rows

table = doc.add_table(nlevels + nrows, ncolumns)
table.style = 'Table Grid'
table.alignment = WD_TABLE_ALIGNMENT.CENTER

# add the header rows.
for i in range(nlevels):
    for j in range(ncolumns):
        text = df.columns[j][i]
        # Skip unnamed cells
        if text[:8] == "Unnamed:":
            continue
        
        if j==0 or text != table.cell(i, j-1).text:
            table.cell(i, j).text = text
            table.cell(i, j).paragraphs[0].paragraph_format.alignment = WD_TABLE_ALIGNMENT.CENTER
        else:
            table.cell(i, j-1).merge(table.cell(i, j))

# add the rest of the data frame
for i in range(nrows):
    for j in range(ncolumns):
        table.cell(i + nlevels, j).text = df.iloc[i, j]
        table.cell(i + nlevels, j).paragraphs[0].paragraph_format.alignment = WD_TABLE_ALIGNMENT.CENTER


In [183]:
# Save the document
root = filename.split('.')[0]
doc.save(os.path.join(directory, f"{root}.docx"))